In [1]:
import pandas
pandas.options.mode.chained_assignment = None
import sys
import numpy as np
from globsML.utils.preprocessing import rescale_data, select_galaxies, create_data_dict, replace_NaN
from globsML.models.skmodels import StandardClassifier

data_path = '../../../data/ACS_sources_original.csv'
data = pandas.read_csv(data_path)
test_galaxies = set(data[data['cluster']=='Virgo']['galaxy'].unique())

In [2]:
df_train, dfs_test = select_galaxies(data, test_galaxies = test_galaxies)
df_train, dfs_test = replace_NaN(df_train, dfs_test)
df = create_data_dict(data, df_train, dfs_test)
df, _ = rescale_data(df)

input_dim = len(df['train']['inputs'][0])

Number of sources in training split after dropping rows with NaN as CI/m/color: 21232
535 sources have been dropped.
NaN values in testing data have been replaced with the correspnding median value observed in the training split


100%|█████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 3829.93it/s]

eccentricity will not be transformed. Skipped.
eccentricity_z will not be transformed. Skipped.


In [3]:
model = StandardClassifier(method='knn', params={'n_neighbors': 12}, data=df)

Loading data...
Fitting model...


In [4]:
model.val()

/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
model.stats_validation

,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,0.929712,0.093458,0.040053,0.943396,0.981311,291,313,30,1062


In [6]:
model.test()

/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/dodo/git/globsML/globsML/utils/eval.py:146: RuntimeWarning: divide by zero encountered in double_scalars
  return auc(x,y)/(2*auc([0,x[-1]], [0,y[-1]])), x, y


In [7]:
model.stats_galaxies

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,VCC1993,0.750000,0.750000,0.036585,0.556307,0.984248,3,4,9,250
1,VCC1407,0.881356,0.187500,0.016304,0.916985,0.992585,52,59,12,795
2,VCC2019,0.941176,0.200000,0.016000,0.948776,0.995206,32,34,8,534
3,VCC1886,0.692308,0.400000,0.008043,0.712735,0.951124,9,13,6,759
4,VCC856,0.816327,0.069767,0.006250,0.944001,0.993197,40,49,3,529
...,...,...,...,...,...,...,...,...,...,...
93,VCC1978,0.929471,0.067004,0.177852,0.902821,0.939747,738,794,53,1092
94,VCC437,0.760000,0.136364,0.007001,0.900799,0.983991,38,50,6,907
95,VCC1178,0.966667,0.093750,0.024523,0.984925,0.996806,87,90,9,457
96,VCC1619,0.934426,0.229730,0.038724,0.904400,0.987938,57,61,17,500


In [8]:
model.stats_galaxies.mean()

/tmp/ipykernel_1806/3415850949.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  model.stats_galaxies.mean()


TPR               0.865176
FDR               0.219195
FPR               0.029691
AUC(FDR,TPR)           inf
AUC(FPR,TPR)      0.980857
# found GCs     115.693878
# total GCs     126.479592
# fake GCs       13.265306
# sources       644.510204
dtype: float64

In [9]:
model.stats_all

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,ALL,0.914724,0.102864,0.025607,0.954329,0.988095,11338,12395,1300,63162
